In [2]:
#load path
import os
import sys
import astropy
from astropy.io import fits
from astropy.table import QTable

import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.stats import sigma_clipped_stats, sigma_clip
from astropy.nddata import NDData
from photutils.psf import extract_stars
from photutils.psf import EPSFBuilder
from astropy.modeling import models, fitting
from photutils.psf.matching import create_matching_kernel
from photutils.psf.matching import TopHatWindow, HanningWindow
from astropy.convolution import convolve_fft

from astropy.wcs import WCS
from photutils.aperture import CircularAperture, aperture_photometry, CircularAnnulus, ApertureStats
from astropy.stats import SigmaClip

from astropy.visualization import (ImageNormalize, MinMaxInterval,PercentileInterval,SqrtStretch)
import glob

import glob
from astropy.io import fits
from astropy.wcs import WCS
from reproject import reproject_interp, reproject_exact
import numpy as np

import warnings


In [3]:
path='./data/'
rawpath=r'/Users/matteo/Google Drive/My Drive/TelescopeData/NEW CAMERA/'
calibpath='./data/Calibs/'
reduxpath='./data/Redux/'
combinepath='./data/Redux/Combined/'
gain=0.25 #e-/ADU at G = 125

Coadding

In [4]:
def measure_fwhm(data, oversampling=1):
    """
    Fits a 2D Gaussian to the data and returns the FWHM in native pixels.
    """
    # Define a grid of coordinates
    y, x = np.mgrid[:data.shape[0], :data.shape[1]]

    # accurate center estimate
    x_cen, y_cen = data.shape[1] // 2, data.shape[0] // 2

    # Initialize a 2D Gaussian model
    g_init = models.Gaussian2D(
        amplitude=np.max(data),
        x_mean=x_cen,
        y_mean=y_cen,
        x_stddev=2,  # initial guess
        y_stddev=2
    )

    # Fit the model to the data
    fit_g = fitting.LevMarLSQFitter()
    g = fit_g(g_init, x, y, data)

    # Calculate FWHM (FWHM = 2.355 * sigma)
    # We take the average of x and y sigma for circular approximation
    sigma_avg = np.min((g.x_stddev.value,g.y_stddev.value))
    axis_ratio = np.max((g.x_stddev.value,g.y_stddev.value)) / np.min((g.x_stddev.value,g.y_stddev.value))
    fwhm_model_pixels = sigma_avg * 2.35482

    # Convert to native image pixels
    fwhm_native = fwhm_model_pixels / oversampling * 0.55

    return fwhm_native, axis_ratio

In [5]:
def build_epfs(image):

    from photutils.detection import find_peaks, DAOStarFinder
    #peaks_tbl = find_peaks(image, threshold=25.0)
    #print('Number of stars found: {}'.format(len(peaks_tbl)))

    dao_finder = DAOStarFinder(20.0,4.0)
    peaks_tbl = dao_finder.find_stars(image)
    print('Number of stars found: {}'.format(len(peaks_tbl)))

    size = 31
    hsize = (size - 1) / 2
    try:
        x = peaks_tbl['xcentroid']
        y = peaks_tbl['ycentroid']
        sharp = peaks_tbl['sharpness']
    except:
        x = peaks_tbl['x_peak']
        y = peaks_tbl['y_peak']
        sharp = np.ones_like(x)

    sat = []
    for ii in range(len(x)):
        sat.append(np.sum(image[int(y[ii]-hsize):int(y[ii]+hsize), int(x[ii]-hsize):int(x[ii]+hsize)]))

    mask = ((x > hsize) & (x < (data.shape[1] -1 - hsize)) & (y > hsize) & (y < (data.shape[0] -1 - hsize)) & np.isfinite(sat))

    stars_tbl = Table()
    stars_tbl['x'] = x[mask]
    stars_tbl['y'] = y[mask]

    print('Number of good stars used for EPSF: {}'.format(len(stars_tbl)))

    nddata = NDData(data=image)
    stars = extract_stars(nddata, stars_tbl, size=size)

    #nrows = 5
    #ncols = 5
    #fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 20),squeeze=True)
    #ax = ax.ravel()
    #for i in range(nrows * ncols):
    #    norm = simple_norm(stars[i], 'log', percent=99.0)
    #    ax[i].imshow(stars[i], norm=norm, origin='lower', cmap='viridis')

    psf_builder = EPSFBuilder(oversampling=1, maxiters=3,progress_bar=False)
    epsf, fitted_stars = psf_builder(stars)

    #from astropy.visualization import simple_norm
    #norm = simple_norm(epsf.data, 'log', percent=99.0)
    #plt.imshow(epsf.data, norm=norm, origin='lower', cmap='viridis')
    #plt.colorbar()

    epsfimage = epsf.data

    fwhm, axis_ratio = measure_fwhm(epsfimage) #fwhm is in arcsec

    return epsfimage, fwhm, axis_ratio


In [6]:
runbootstrap = True

# List of filters to process
filters = ['R','G','I','SII','OIII','Ha','Hb']
targname = 'PSZ2G114.79-33.71'
outsize = 4500
filters=['R']

shortexp=False

for filter in filters:
    # Find all sky-subtracted images for this filter
    images = glob.glob(reduxpath + f'/*{targname}_{filter}*sky_sub.fits')
    images.sort()
    if len(images) == 0:
        print('No images found')
        continue
    else:
        print(f'Processing {len(images)} images for filter {filter}')

    #Generate an header of 5000x5000 pixels with WCS centered on the target and pixel scale of 0.55 arcsec/pixel
    # Use the first image as reference
    ref_hdu = fits.open(images[0])
    ref_data = ref_hdu[0].data
    ref_header = ref_hdu[0].header

    out_header = fits.Header()
    for key in ['CTYPE1','CTYPE2','CRVAL1','CRVAL2','CRPIX1','CRPIX2','CD1_1','CD1_2','CD2_1','CD2_2','FUNIT','EXPTIME']:
        out_header[key] = ref_header[key]

    from astroquery.simbad import Simbad
    result_table = Simbad.query_object(targname)
    if result_table is None:
        print('Could not find target {} in Simbad, using default RA and DEC'.format(targname))
    else:
        print('Query succeeded for target {}'.format(targname))
        try:
         #Find object RA and DEC
         center_ra = result_table['ra'][0]
         center_dec = result_table['dec'][0]
         print('Target coordinates: {:7.4f} - {:7.4f} '.format(center_ra, center_dec))
         out_header['CTYPE1'] = 'RA---TAN'
         out_header['CTYPE2'] = 'DEC--TAN'
         out_header['CRVAL1'] = center_ra
         out_header['CRVAL2'] = center_dec
         out_header['CRPIX1'] = outsize/2.
         out_header['CRPIX2'] = outsize/2.
         out_header['CD1_1'] = -0.00015355555
         out_header['CD2_2'] = 0.00015355555
         out_header['CD1_2'] = 0
         out_header['CD2_1'] = 0
         ref_data = np.zeros((outsize,outsize))
        except:
         print('Could not find target {} in Simbad, using default RA and DEC'.format(targname))

    out_wcs = WCS(out_header)
    stack = []
    stackvar = []

    cnt = 0

    for img in images:
      with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        hdu = fits.open(img)
        print(' Now reading ({}): {}'.format(cnt,img))
        cnt+=1
        data = hdu[0].data
        var  = hdu[1].data
        data[np.isnan(var)] = np.nan
        header = hdu[0].header
        wcs = WCS(header)
        exptime = hdu[0].header['EXPTIME']
        if shortexp:
          if exptime>30:
            print(f'  Skipping image {img} with exptime {exptime}s')
            continue
        else:
          if exptime<30:
            print(f'  Skipping image {img} with exptime {exptime}s')
            continue
        # Reproject to reference WCS
        reproj_data, _ = reproject_interp((data, wcs), out_wcs, shape_out=ref_data.shape, parallel=4, order='nearest-neighbor')
        reproj_var,  _ = reproject_interp((var,  wcs), out_wcs, shape_out=ref_data.shape, parallel=4, order='nearest-neighbor')
        #print(np.nanvar(reproj_data[2050:2150,2700:2800]),np.nanmedian(reproj_var[2050:2150,2700:2800]))
        stack.append(reproj_data)
        stackvar.append(reproj_var)

    # Combine (with sigma clipped mean statistics)
    print('Stacking ....')
    stack = np.array(stack)
    stackvar = np.array(stackvar)

    clip = sigma_clip(stack, axis=0, masked=False)

    #tmphdu = fits.PrimaryHDU(clip, out_header)
    #tmphdu.writeto(combinepath + f'/TMPclip.fits', overwrite=True)

    varclip = np.copy(stackvar)
    varclip[np.isnan(clip)] = np.nan

    #tmphdu = fits.PrimaryHDU(varclip, out_header)
    #tmphdu.writeto(combinepath + f'/TMPclipvar.fits', overwrite=True)

    combined = np.nanmean(clip, axis=0)
    expmap = np.nansum(np.isfinite(varclip),axis=0)
    combinedvar = np.nansum(varclip, axis=0)/expmap**2

    nans = np.isnan(combined)

    if runbootstrap:

       sumdata = np.zeros_like(combined)
       sumqdata = np.zeros_like(combined)
       Nboot = 100

       bootimas = []

       print('Bootstrapping images...')

       for boot in range(Nboot):
           indices = np.random.choice(len(clip), size=len(clip), replace=True)
           bootima = np.nanmean(clip[indices,...], axis=0)
           sumdata += bootima
           sumqdata += bootima**2
           bootimas.append(bootima)

       vardata = sumqdata/Nboot - (sumdata/Nboot)**2
       #vardata2 = np.nanvar(clip, axis=0)/(len(clip))

    else:
        vardata = np.nanvar(clip, axis=0)/expmap

    vardata[nans] = np.nan

    epsf, fwhm, axrat = build_epfs(combined)

    out_header['NEXP'] = len(stack)
    out_header['PSFFWHM'] = fwhm
    out_header['PSFAXRAT'] = axrat

    # Save combined image and uncertainty
    hdu1 = fits.PrimaryHDU(combined.astype(np.float32), out_header)
    hdu2 = fits.ImageHDU(combinedvar.astype(np.float32), out_header)
    hdu3 = fits.ImageHDU(vardata.astype(np.float32), out_header)
    hdu5 = fits.ImageHDU(expmap.astype(np.float32), out_header)
    hdu4 = fits.ImageHDU(epsf, out_header)
    hduout = fits.HDUList([hdu1, hdu2, hdu3, hdu5, hdu4])

    if shortexp:
      hduout.writeto(combinepath + f'/Combined_{targname}_{filter}_shortexp.fits', overwrite=True)
    else:
      hduout.writeto(combinepath + f'/Combined_{targname}_{filter}.fits', overwrite=True)

Processing 79 images for filter G
Query succeeded for target PSZ2G114.79-33.71
Target coordinates:  5.1614 - 28.6946 
 Now reading (0): ./data/Redux/2025-11-25_19-08-15_sci_PSZ2G114.79-33.71_G_exp300.00_0000_sky_sub.fits
 Now reading (1): ./data/Redux/2025-11-25_19-13-28_sci_PSZ2G114.79-33.71_G_exp300.00_0001_sky_sub.fits
 Now reading (2): ./data/Redux/2025-11-25_19-19-16_sci_PSZ2G114.79-33.71_G_exp300.00_0002_sky_sub.fits
 Now reading (3): ./data/Redux/2025-11-25_19-24-30_sci_PSZ2G114.79-33.71_G_exp300.00_0003_sky_sub.fits
 Now reading (4): ./data/Redux/2025-11-25_19-30-10_sci_PSZ2G114.79-33.71_G_exp300.00_0004_sky_sub.fits
 Now reading (5): ./data/Redux/2025-11-25_19-35-56_sci_PSZ2G114.79-33.71_G_exp300.00_0005_sky_sub.fits
 Now reading (6): ./data/Redux/2025-11-25_19-41-23_sci_PSZ2G114.79-33.71_G_exp300.00_0006_sky_sub.fits
 Now reading (7): ./data/Redux/2025-11-25_19-46-59_sci_PSZ2G114.79-33.71_G_exp300.00_0007_sky_sub.fits
 Now reading (8): ./data/Redux/2025-11-25_19-53-02_sci_PSZ

/var/folders/ls/6hrs6lfd0h3_n2gkfnmq_k940000gn/T/ipykernel_93530/491221708.py:105: RuntimeWarning: Mean of empty slice
  combined = np.nanmean(clip, axis=0)
/var/folders/ls/6hrs6lfd0h3_n2gkfnmq_k940000gn/T/ipykernel_93530/491221708.py:107: RuntimeWarning: invalid value encountered in divide
  combinedvar = np.nansum(varclip, axis=0)/expmap**2


Bootstrapping images...


/var/folders/ls/6hrs6lfd0h3_n2gkfnmq_k940000gn/T/ipykernel_93530/491221708.py:123: RuntimeWarning: Mean of empty slice
  bootima = np.nanmean(clip[indices,...], axis=0)


Number of stars found: 49
Number of good stars used for EPSF: 26


In [ ]:
imatest = fits.open(combinepath + f'/Combined_PSZ2G114.79-33.71_R.fits')[0].data
print(build_epfs(imatest))